In [4]:
#Grab Data
import yfinance as yf

#Usual Suspects
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# plt.style.use("seaborn-v0_8-deep")

import plotly.express as px
from pypfopt.plotting import plot_weights
import seaborn as sns

# Use PyPortfolioOpt for Calculations
from pypfopt import EfficientFrontier, objective_functions
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel, plotting
from pypfopt import DiscreteAllocation

In [5]:
#Create a Portfolio
df = pd.read_csv('hackathon_sample_v2.csv')
symbols = df.stock_ticker.unique()
symbols = symbols[~pd.isnull(symbols)]
symbols = symbols.tolist()[:100]

[**                     5%                       ]  177 of 3565 completed

In [6]:
porfolioFile = "yfinance_download.csv"

if not os.path.isfile(porfolioFile):
    portfolio = yf.download(symbols, start="2000-01-01", end="2023-12-31")['Adj Close']
    portfolio.to_csv(porfolioFile, index=True)
else:
    portfolio = pd.read_csv(porfolioFile)

portfolio.head()


[*********************100%***********************]  100 of 100 completed

34 Failed downloads:
['FVB', 'SVEV', 'EVVV', 'ASD', 'CIMA', 'MESG', 'DNDN', 'VVTV', 'EMLX', 'ARO', 'OLDB', 'DZB', 'LHSG', 'SCAI', 'ABSC', 'SCNT', 'PHLY', 'ASKJ', 'CXG', 'GZTC', 'CKFR']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2000-01-01 -> 2023-12-31)')
['TMBR', 'ACIA', 'EIGI', 'ADLA', 'EXPR', 'WE', 'SCGW', 'HCR', 'RICE', 'IDTI', 'SGFY', 'AGIL', 'RE']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Ticker,ABM,ABSC,ACIA,ADLA,AFRM,AGIL,ALGT,AMT,ARO,ASD,...,TGNT,TMBR,TPR,TRGP,UPR,VRTS,VVTV,VVV,WE,WEX
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03 00:00:00+00:00,5.837878,NaN,NaN,NaN,NaN,NaN,NaN,22.544762,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04 00:00:00+00:00,5.855843,NaN,NaN,NaN,NaN,NaN,NaN,22.304422,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05 00:00:00+00:00,5.783989,NaN,NaN,NaN,NaN,NaN,NaN,22.977396,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06 00:00:00+00:00,5.801954,NaN,NaN,NaN,NaN,NaN,NaN,22.737043,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07 00:00:00+00:00,5.819914,NaN,NaN,NaN,NaN,NaN,NaN,23.986858,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


[***************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [7]:
#SP500 ETF Benchmark
market_prices = yf.download("SPY", start='2000-01-01', end='2023-12-31')["Adj Close"]
market_prices.head()

[*********************100%***********************]  1 of 1 completed


KeyError: 'SPY'

In [ ]:
marketCapsFile = 'market_caps.csv'
mcaps = {}

if not os.path.isfile(marketCapsFile):
    mcaps = {}
    for t in symbols:
        try:
            stock = yf.Ticker(t)
            mcaps[t] = stock.info["marketCap"]
        except:
            pass
    market_caps = pd.Series(mcaps)
    market_caps.to_csv(marketCapsFile, index=False)
else:
    mcaps = pd.read_csv(marketCapsFile)

In [ ]:
mcaps

# Getting Priors

In [ ]:
#Calculate Sigma and Delta to get implied market returns
#Ledoit-Wolf is a particular form of shrinkage, where the shrinkage coefficient is computed using O?
S = risk_models.CovarianceShrinkage(portfolio).ledoit_wolf()

delta = black_litterman.market_implied_risk_aversion(market_prices)
delta

In [ ]:
#Visualize the Covariant Correlation
sns.heatmap(S.corr(), cmap='coolwarm')

In [ ]:
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)
market_prior

In [ ]:
#What am I looking at here?
market_prior.plot.barh(figsize=(10,5));

# Integrating Views

In [32]:
#You don't have to provide views on all the assets
viewdict = {
    'AAPL':0.10,
    'MSFT':0.10,
    'META':0.05,
    'AMZN':0.30,
    'XOM':0.02,
    'UNH':0.01,
    'JNJ':0.15,
    'V':0.09,
    'HD':0.16,
    'ABBV':0.07,
    'KO':0.01,
    'DIS':-0.23,
    'T':0.16,
    'UPS':0.10,
    'LMT':-0.09,
    'CAT':0.30,
    'F':0.16,
    'MAR':-0.08,
    'O':0.30,
    'HSY':-0.26
}

bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict)

## Creating Confidences

In [33]:
intervals = [
    (0, 0.25),
    (0.1, 0.4),
    (-0.1, 0.15),
    (-0.05, 0.1),
    (0.15, 0.25),
    (-0.1, 0),
    (0.1, 0.2),
    (0.08, 0.12),
    (0.1, 0.9),
    (0, 0.3),
    (0, 0.25),
    (0.1, 0.4),
    (-0.1, 0.15),
    (-0.05, 0.1),
    (0.15, 0.25),
    (-0.1, 0),
    (0.1, 0.2),
    (0.08, 0.12),
    (0.1, 0.9),
    (0, 0.3),
]

In [ ]:
variances = []
for lb, ub in intervals:
    sigma = (ub - lb)/2
    variances.append(sigma ** 2)

print(variances)
omega = np.diag(variances)

# Calculate Posterior Estimate Returns

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
im = ax.imshow(omega)

# We want to show all ticks...
ax.set_xticks(np.arange(len(bl.tickers)))
ax.set_yticks(np.arange(len(bl.tickers)))

ax.set_xticklabels(bl.tickers)
ax.set_yticklabels(bl.tickers)
plt.show()

In [36]:
# We are using the shortcut to automatically compute market-implied prior
bl = BlackLittermanModel(S, pi="market", market_caps=mcaps, risk_aversion=delta,
                        absolute_views=viewdict, omega=omega)

In [ ]:
# Posterior estimate of returns
ret_bl = bl.bl_returns()
ret_bl 

In [ ]:
rets_df = pd.DataFrame([market_prior, ret_bl, pd.Series(viewdict)],
             index=["Prior", "Posterior", "Views"]).T
rets_df

In [ ]:
rets_df.plot.bar(figsize=(12,8));

In [ ]:
S_bl = bl.bl_cov()
plotting.plot_covariance(S_bl);

# Portfolio Allocation

In [ ]:
ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()
weights

In [ ]:
pd.Series(weights).plot.pie(figsize=(9,9));

In [ ]:
# Maximum Sharpe
ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()

plot_weights(weights)
ef.portfolio_performance(verbose = True, risk_free_rate = 0.009)